In [1]:
#| default_exp models.tsmixer

In [2]:
#| hide
%load_ext autoreload
%autoreload 2

In [3]:
#| export

import torch
import torch.nn as nn
import torch.nn.functional as F

from neuralforecast.losses.pytorch import MAE
from neuralforecast.common._base_multivariate import BaseMultivariate

In [58]:
#| hide
class TSMixer(BaseMultivariate):
    """ TSMixer

    TODO documentation here

    **Parameters:**<br>
    `h`: int, Forecast horizon. <br>
    `input_size`: int, autorregresive inputs size, y=[1,2,3,4] input_size=2 -> y_[t-2:t]=[1,2].<br>
    `n_series`: int, number of time-series.<br>
    `stat_exog_list`: str list, static exogenous columns.<br>
    `hist_exog_list`: str list, historic exogenous columns.<br>
    `futr_exog_list`: str list, future exogenous columns.<br>
    `n_stacks`: int=2, number of stacks in the model.<br>
    `multi_layer`: int=5, multiplier for FC hidden size on StemGNN blocks.<br>
    `dropout_rate`: float=0.5, dropout rate.<br>
    `leaky_rate`: float=0.2, alpha for LeakyReLU layer on Latent Correlation layer.<br>
    `loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
    `valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
    `max_steps`: int=1000, maximum number of training steps.<br>
    `learning_rate`: float=1e-3, Learning rate between (0, 1).<br>
    `num_lr_decays`: int=-1, Number of learning rate decays, evenly distributed across max_steps.<br>
    `early_stop_patience_steps`: int=-1, Number of validation iterations before early stopping.<br>
    `val_check_steps`: int=100, Number of training steps between every validation loss check.<br>
    `batch_size`: int, number of windows in each batch.<br>
    `step_size`: int=1, step size between each window of temporal data.<br>
    `scaler_type`: str='robust', type of scaler for temporal inputs normalization see [temporal scalers](https://nixtla.github.io/neuralforecast/common.scalers.html).<br>
    `random_seed`: int, random_seed for pytorch initializer and numpy generators.<br>
    `num_workers_loader`: int=os.cpu_count(), workers to be used by `TimeSeriesDataLoader`.<br>
    `drop_last_loader`: bool=False, if True `TimeSeriesDataLoader` drops last non-full batch.<br>
    `alias`: str, optional,  Custom name of the model.<br>
    `**trainer_kwargs`: int,  keyword trainer arguments inherited from [PyTorch Lighning's trainer](https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.trainer.trainer.Trainer.html?highlight=trainer).<br>    
    """
    # Class attributes
    SAMPLING_TYPE = 'multivariate'
    
    def __init__(self,
                 h,
                 input_size,
                 n_series,
                 futr_exog_list = None,
                 hist_exog_list = None,
                 stat_exog_list = None,
                 n_stacks = 2,
                 multi_layer: int = 5,
                 dropout_rate: float = 0.5,
                 leaky_rate: float = 0.2,
                 loss = MAE(),
                 valid_loss = None,
                 max_steps: int = 1000,
                 learning_rate: float = 1e-3,
                 num_lr_decays: int = 3,
                 early_stop_patience_steps: int =-1,
                 val_check_steps: int = 100,
                 batch_size: int = 32,
                 step_size: int = 1,
                 scaler_type: str = 'robust',
                 random_seed: int = 1,
                 num_workers_loader = 0,
                 drop_last_loader = False,
                 **trainer_kwargs):

        # Inherit BaseMultivariate class
        super(TSMixer, self).__init__(h=h,
                                      input_size=input_size,
                                      n_series=n_series,
                                      futr_exog_list=futr_exog_list,
                                      hist_exog_list=hist_exog_list,
                                      stat_exog_list=stat_exog_list,                                    
                                      loss=loss,
                                      valid_loss=valid_loss,
                                      max_steps=max_steps,
                                      learning_rate=learning_rate,
                                      num_lr_decays=num_lr_decays,
                                      early_stop_patience_steps=early_stop_patience_steps,
                                      val_check_steps=val_check_steps,
                                      batch_size=batch_size,
                                      step_size=step_size,
                                      scaler_type=scaler_type,
                                      num_workers_loader=num_workers_loader,
                                      drop_last_loader=drop_last_loader,
                                      random_seed=random_seed,
                                      **trainer_kwargs)


    def res_block(self, inputs, batch_size, input_size, num_channels):
        self.batch_norm = nn.BatchNorm2d(num_features=num_channels)
        self.linear = nn.Linear(in_features=input_size, out_features=input_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
    
        # time block
        x = nn.BatchNorm2d(num_features=num_channels)
        x = torch.transpose(x, 3, 2)
        x = nn.Linear(in_features=num_channels, out_features=num_channels)(x)
        x = nn.ReLU(x)
        x = nn.Dropout(0.1)(x)
        res = inputs + x

        # feature block
        x = torch.transpose(res, 3, 2)
        x = nn.BatchNorm2d(num_features=num_channels)(x)
        x = nn.Linear(in_features=num_channels, out_features=num_channels)(x)
        x = nn.ReLU(x)
        x = nn.Dropout(0.1)(x)
        x = nn.Linear(in_features=num_channels, out_features=num_channels)(x)
        x = nn.Dropout(0.1)(x)
        return x + res

    def forward(self, windows_batch):
        inputs = windows_batch['insample_y']
        batch_size = windows_batch.shape[0]
        input_size = windows_batch.shape[1]
        num_channels = windows_batch.shape[2]
        horizon = self.h

        layers = nn.ModuleList(
            [# TODO loop this based on `num_stacks`
             self.res_block(inputs=inputs,
                            batch_size=batch_size,
                            input_size=input_size,
                            num_channels=num_channels),
             torch.transpose(3, 2),
             nn.Linear(in_features=input_size, out_features=horizon),
             torch.transpose(3, 2)]
            )
        
        for i, l in enumerate(layers):
            if i == 0:
                x = l(inputs)
            else:
                x = (l(x))

        return x


In [181]:
#| eval: false
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.utils import AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.pytorch import MAE

Y_train_df = AirPassengersPanel[AirPassengersPanel.ds<AirPassengersPanel['ds'].values[-12]].reset_index(drop=True) # 132 train
Y_test_df = AirPassengersPanel[AirPassengersPanel.ds>=AirPassengersPanel['ds'].values[-12]].reset_index(drop=True) # 12 test

In [188]:
#| export
class TSMixer(BaseMultivariate):
    # Class attributes
    SAMPLING_TYPE = 'multivariate'
    
    def __init__(self,
                 h,
                 input_size,
                 n_series,
                 futr_exog_list = None,
                 hist_exog_list = None,
                 stat_exog_list = None,
                 n_stacks = 2,
                 multi_layer: int = 5,
                 dropout_rate: float = 0.5,
                 leaky_rate: float = 0.2,
                 loss = MAE(),
                 valid_loss = None,
                 max_steps: int = 1000,
                 learning_rate: float = 1e-3,
                 num_lr_decays: int = 3,
                 early_stop_patience_steps: int =-1,
                 val_check_steps: int = 100,
                 batch_size: int = 32,
                 step_size: int = 1,
                 scaler_type: str = 'robust',
                 random_seed: int = 1,
                 num_workers_loader = 0,
                 drop_last_loader = False,
                 **trainer_kwargs):

        # Inherit BaseMultivariate class
        super(TSMixer, self).__init__(h=h,
                                      input_size=input_size,
                                      n_series=n_series,
                                      futr_exog_list=futr_exog_list,
                                      hist_exog_list=hist_exog_list,
                                      stat_exog_list=stat_exog_list,                                    
                                      loss=loss,
                                      valid_loss=valid_loss,
                                      max_steps=max_steps,
                                      learning_rate=learning_rate,
                                      num_lr_decays=num_lr_decays,
                                      early_stop_patience_steps=early_stop_patience_steps,
                                      val_check_steps=val_check_steps,
                                      batch_size=batch_size,
                                      step_size=step_size,
                                      scaler_type=scaler_type,
                                      num_workers_loader=num_workers_loader,
                                      drop_last_loader=drop_last_loader,
                                      random_seed=random_seed,
                                      **trainer_kwargs)

        # Exogenous variables
        self.futr_input_size = len(self.futr_exog_list)
        self.hist_input_size = len(self.hist_exog_list)
        self.stat_input_size = len(self.stat_exog_list)

        self.unit = n_series
        self.stack_cnt = n_stacks
        self.alpha = leaky_rate
        self.time_step = input_size
        self.horizon = h
        self.dropout_rate = dropout_rate
        self.input_size = input_size
        self.num_channels = n_series
        self.H = H

        self.batch_norm = nn.BatchNorm2d(num_features=self.num_channels)
        self.linear = nn.Linear(in_features=H, out_features=H)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)

    def forward(self, windows_batch, *args, **kwargs):
        inputs = windows_batch['insample_y']
        print('raw', inputs.shape)
        
        inputs = inputs.unsqueeze(1).permute(0, 3, 2, 1)
        print('unsqueeze', inputs.shape)
        
        for _ in range(0, 1): # self.stack_cnt
            # time block
            x = nn.BatchNorm2d(num_features=self.num_channels)(inputs)
            print('batch norm', x.shape)

            x = torch.transpose(x, 3, 2)
            print('transpose', x.shape)
            
            x = nn.Linear(in_features=self.input_size, out_features=self.input_size)(x)
            print('linear', x.shape)
            
            x = nn.ReLU()(x)
            print('relu', x.shape)

            x = nn.Dropout(p=self.dropout_rate)(x)
            print('dropout', x.shape)

            res = inputs.permute(0, 1, 3, 2) + x
            print('res', res.shape)
            print('--------')
            print('\n')
            
            # feature block
            x = torch.transpose(res, 3, 2)
            print('feature transpose', x.shape)

            x = nn.BatchNorm2d(num_features=self.num_channels)(x)
            print('batch norm', x.shape)

            x = nn.Linear(in_features=self.input_size, out_features=self.input_size)(x)
            print('linear', x.shape)

            x = nn.ReLU()(x)
            print('relu', x.shape)

            x = nn.Dropout(p=self.dropout_rate)(x)
            print('dropout', x.shape)

            x = nn.Linear(in_features=self.input_size, out_features=self.input_size)(x)
            print('linear', x.shape)

            x = nn.Dropout(p=self.dropout_rate)(x)
            print('dropout', x.shape)

            x = x + res
            print('final x', x.shape)

        # temporal projection
        x = torch.transpose(res, 3, 2)
        x = nn.Linear(in_features=self.input_size, out_features=self.horizon)(x)
        x = torch.transpose(x, 3, 2)
        return x

In [189]:
model = TSMixer(h=12,
                input_size=24,
                n_series=2,
                stat_exog_list=['airline1'],
                futr_exog_list=['trend'],
                scaler_type='robust',
                max_steps=200,
                early_stop_patience_steps=-1,
                val_check_steps=10,
                learning_rate=1e-3,
                loss=None,
                valid_loss=None,
                batch_size=32
                )

list(model.parameters())

fcst = NeuralForecast(models=[model], freq='M')
fcst.fit(df=Y_train_df, static_df=AirPassengersStatic, val_size=12)
forecasts = fcst.predict(futr_df=Y_test_df)

Seed set to 1


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

raw torch.Size([1, 24, 2])
unsqueeze torch.Size([1, 2, 24, 1])
batch norm torch.Size([1, 2, 24, 1])
transpose torch.Size([1, 2, 1, 24])
linear torch.Size([1, 2, 1, 24])
relu torch.Size([1, 2, 1, 24])
dropout torch.Size([1, 2, 1, 24])
res torch.Size([1, 2, 1, 24])
--------


feature transpose torch.Size([1, 2, 24, 1])
batch norm torch.Size([1, 2, 24, 1])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (48x1 and 24x24)